# 5. Crosstabs and aggregations

Tally has a powerful aggregation engine, which allows us to examine our data at every step of the way. 

Tally's aggregation engine supports
- weights
- significance testing
- statistics (mean, stddev, sum, etc)
- nesting
- filters (using logic to select rows to aggregate)
- arrays (i.e. grids/loops)

In [ ]:
#
# In order to run this notebook, you first have to install Tally. To install tally you need a token that gives you access.
#
from google.colab import files
import json
import io
import os
# Check if the file 'tally_keys.json' exists
if not os.path.exists('tally_keys.json'):
  uploaded = files.upload()
  # Assuming only one file is uploaded, get its filename and content
  filename = list(uploaded.keys())[0]
  file_content = uploaded[filename]
  # Load JSON directly from the uploaded content
  keys = json.loads(file_content.decode('utf-8'))
else:
  # If the file already exists, just load its content
  with open('tally_keys.json', 'r') as f:
      keys = json.load(f)

try:
  # Try to import the package
  import example_package
except ImportError:
  # If the import fails, the package is not installed. Install it.
  !pip install git+https://{keys['tally_api']}@github.com/datasmoothie/tally-core.git@master

In [1]:
import tally_core as tc
import pandas as pd
import json
dataset = tc.DataSet('Museum')

dataset.read_gvn('./data/Example Data (A).json', './data/Example Data (A).parquet')

## Selecting the contents of a crosstab
We will use the Sport store test dataset. To explore our crosstabs we will use the variable `q1` ("what is your main fitness activity") along with `gender` and `locality`.

The parameters available for the crosstab method are:

- `x` (required)
- `y` - default []
- `w` - default None
- `f` - default None
- `ci` - default counts
- `base` - default auto
- `stats` - default None
- `sig_level` - default None
- `decimals` - default 1
- `xtotal` - default False
- `text_key` - default None


### Side/stub variables

The only required parameter is the x variable, often called the "side" or "stub" variable. By default, this will show the number of responses for each answers, i.e. the count.

In [2]:
dataset.crosstab(x='q1')

Question                                                                         Total
Values                                                                           Total
Question                                Values                                        
q1. What is your main fitness activity? Base                                    8255.0
                                        Swimming                                 297.0
                                        Running/jogging                          397.0
                                        Lifting weights                         2298.0
                                        Aerobics                                2999.0
                                        Yoga                                     194.0
                                        Pilates                                  477.0
                                        Football (soccer)                        894.0
                                        Basketball                               131.0
                                        Hockey                                     4.0
                                        Other                                     91.0
                                        I regularly change my fitness activity   104.0
                                        Not applicable - I don't exercise        369.0

### Top/banner variables
We provide the `y` parameter to get "top" or "banner" variables.

In [3]:
dataset.crosstab(x='q1', y=['gender', 'locality'])

Question                                                                       gender. What is your gender?         locality. How would you describe the areas in which you live?                               
Values                                                                                                 Male  Female                               CBD (central business district)   Urban Suburban  Rural Remote
Question                                Values                                                                                                                                                                  
q1. What is your main fitness activity? Base                                                         3952.0  4303.0                                             3106.0             2245.0   1180.0  718.0  829.0
                                        Swimming                                                      145.0   152.0                                              140.0               72.0     38.0   15.0   25.0
                                        Running/jogging                                               205.0   192.0                                              156.0              104.0     55.0   29.0   45.0
                                        Lifting weights                                              1094.0  1204.0                                              835.0              671.0    320.0  175.0  243.0
                                        Aerobics                                                     1438.0  1561.0                                             1200.0              767.0    435.0  270.0  271.0
                                        Yoga                                                           95.0    99.0                                               70.0               49.0     33.0   18.0   19.0
                                        Pilates                                                       199.0   278.0                                              154.0              127.0     82.0   47.0   52.0
                                        Football (soccer)                                             447.0   447.0                                              315.0              241.0    131.0   87.0   98.0
                                        Basketball                                                     65.0    66.0                                               44.0               35.0     24.0   15.0   12.0
                                        Hockey                                                          3.0     1.0                                                2.0                1.0      1.0    0.0    0.0
                                        Other                                                          41.0    50.0                                               37.0               24.0     13.0    9.0    6.0
                                        I regularly change my fitness activity                         39.0    65.0                                               45.0               22.0      9.0   10.0   16.0
                                        Not applicable - I don't exercise                             181.0   188.0                                              108.0              132.0     39.0   43.0   42.0

### Weight
We can apply a weight variable to the crosstab with `w`.

In [4]:
dataset.crosstab(x='q1', y='gender', w='weight_a')

Question                                                                       gender. What is your gender?        
Values                                                                                                 Male  Female
Question                                Values                                                                     
q1. What is your main fitness activity? Base                                                         3970.0  4284.0
                                        Swimming                                                      142.3   139.8
                                        Running/jogging                                               181.2   185.8
                                        Lifting weights                                              1040.3  1119.6
                                        Aerobics                                                     1545.2  1647.3
                                        Yoga                                                          107.2   112.2
                                        Pilates                                                       188.8   297.6
                                        Football (soccer)                                             462.7   471.7
                                        Basketball                                                     54.3    64.4
                                        Hockey                                                          4.9     0.5
                                        Other                                                          40.0    45.9
                                        I regularly change my fitness activity                         43.8    58.7
                                        Not applicable - I don't exercise                             160.0   140.8

### Filter (case selection)

We can select a subset of the data to run the crosstab on with the `f` variable. The filter understand complex [Tally logic operators](tally_logic). Here, we set the filter to only show `gender` answers with code 2 (female).

In [5]:
dataset.crosstab(x='q1', y='gender', f={'gender':[2]})

Question                                                                       gender. What is your gender?        
Values                                                                                                 Male  Female
Question                                Values                                                                     
q1. What is your main fitness activity? Base                                                            0.0  4303.0
                                        Swimming                                                        0.0   152.0
                                        Running/jogging                                                 0.0   192.0
                                        Lifting weights                                                 0.0  1204.0
                                        Aerobics                                                        0.0  1561.0
                                        Yoga                                                            0.0    99.0
                                        Pilates                                                         0.0   278.0
                                        Football (soccer)                                               0.0   447.0
                                        Basketball                                                      0.0    66.0
                                        Hockey                                                          0.0     1.0
                                        Other                                                           0.0    50.0
                                        I regularly change my fitness activity                          0.0    65.0
                                        Not applicable - I don't exercise                               0.0   188.0

### Cell contents
We can select what to show in the cells, using `counts`, `c%`, and `r%` for counts, column percentages and row percentages.

In [6]:
dataset.crosstab(x='q1', y='gender', ci=['counts', 'r%'])

Question                                                                       gender. What is your gender?        
Values                                                                                                 Male  Female
Question                                Values                                                                     
q1. What is your main fitness activity? Base                                                         3952.0  4303.0
                                        Swimming                                                      145.0   152.0
                                                                                                       48.8    51.2
                                        Running/jogging                                               205.0   192.0
                                                                                                       51.6    48.4
                                        Lifting weights                                              1094.0  1204.0
                                                                                                       47.6    52.4
                                        Aerobics                                                     1438.0  1561.0
                                                                                                       47.9    52.1
                                        Yoga                                                           95.0    99.0
                                                                                                       49.0    51.0
                                        Pilates                                                       199.0   278.0
                                                                                                       41.7    58.3
                                        Football (soccer)                                             447.0   447.0
                                                                                                       50.0    50.0
                                        Basketball                                                     65.0    66.0
                                                                                                       49.6    50.4
                                        Hockey                                                          3.0     1.0
                                                                                                       75.0    25.0
                                        Other                                                          41.0    50.0
                                                                                                       45.1    54.9
                                        I regularly change my fitness activity                         39.0    65.0
                                                                                                       37.5    62.5
                                        Not applicable - I don't exercise                             181.0   188.0
                                                                                                       49.1    50.9

### Base
When a crosstab is weighted, we can select what bases we want to show in the crosstab. The options are `weighted`, `unweighted`, `both`, and `auto`. Auto will only show one weight.

In [7]:
dataset.crosstab('q1', w='weight_a', base='both')

Question                                                                         Total
Values                                                                           Total
Question                                Values                                        
q1. What is your main fitness activity? Unweighted base                         8255.0
                                        Base                                    8255.0
                                        Swimming                                 282.0
                                        Running/jogging                          367.0
                                        Lifting weights                         2159.9
                                        Aerobics                                3192.5
                                        Yoga                                     219.4
                                        Pilates                                  486.5
                                        Football (soccer)                        934.4
                                        Basketball                               118.6
                                        Hockey                                     5.4
                                        Other                                     86.0
                                        I regularly change my fitness activity   102.5
                                        Not applicable - I don't exercise        300.8

### Stats

In [20]:
dataset.crosstab('q1', 'gender', stats=['mean', 'sem', 'stddev', 'varcoeff', 'min', 'max', 'median', 'lower_q', 'upper_q'])

Question                                                                       gender. What is your gender?        
Values                                                                                                 Male  Female
Question                                Values                                                                     
q1. What is your main fitness activity? Base                                                         3952.0  4303.0
                                        Swimming                                                      145.0   152.0
                                        Running/jogging                                               205.0   192.0
                                        Lifting weights                                              1094.0  1204.0
                                        Aerobics                                                     1438.0  1561.0
                                        Yoga                                                           95.0    99.0
                                        Pilates                                                       199.0   278.0
                                        Football (soccer)                                             447.0   447.0
                                        Basketball                                                     65.0    66.0
                                        Hockey                                                          3.0     1.0
                                        Other                                                          41.0    50.0
                                        I regularly change my fitness activity                         39.0    65.0
                                        Not applicable - I don't exercise                             181.0   188.0
                                        Mean                                                           10.3    10.7
                                        Std. err. of mean                                               0.4     0.4
                                        Std. dev                                                       23.5    24.2
                                        Coefficient of variation                                        2.3     2.3
                                        Min                                                             1.0     1.0
                                        Max                                                            99.0    99.0
                                        Median                                                          4.0     4.0
                                        Lower quartile                                                  3.0     3.0
                                        Upper quartile                                                  5.0     6.0

### Significance testing

Significance testing is done by adding a value for the alpha parameter in column proportional testing. For alpha=0.05 we do

In [24]:
dataset.crosstab('q2b', 'gender', sig_level=0.05, stats=['mean'])

Question                                                       gender. What is your gender?        
Values                                                                                 Male  Female
Test-IDs                                                                                  A       B
Question                                           Values                                          
q2b. How regularly do you participate in any fi... Base                              2998.0  3309.0
                                                   Regularly                          385.0   356.0
                                                   0.05                                   B     NaN
                                                   Irregularly                       2458.0  2689.0
                                                   0.05                                 NaN     NaN
                                                   Never                              155.0   264.0
                                                   0.05                                 NaN       A
                                                   Mean                                 1.9     2.0
                                                   0.05                                 NaN       A

The header gets a new row, Test-ID, and each column gets a letter as an ID. In the table above, Men are significantly more likely to work out regularly than women, and women are significantly liklier to never work out than men. Note that this is a test dataset, not real data!

### Decimals

Crosstabs can have as many decimals as we like, and this is controlled with the `decimals` parameter.

In [28]:
dataset.crosstab('q2b', 'gender', ci=['c%'], decimals=4)

Question                                                       gender. What is your gender?           
Values                                                                                 Male     Female
Question                                           Values                                             
q2b. How regularly do you participate in any fi... Base                           2998.0000  3309.0000
                                                   Regularly                        12.8419    10.7585
                                                   Irregularly                      81.9880    81.2632
                                                   Never                             5.1701     7.9782

### Totals

We can add a column at the far left that shows the total for the dataset.

In [29]:
dataset.crosstab('q2b', 'gender', xtotal=True)

Question                                                         Total gender. What is your gender?        
Values                                                           Total                         Male  Female
Question                                           Values                                                  
q2b. How regularly do you participate in any fi... Base         6307.0                       2998.0  3309.0
                                                   Regularly     741.0                        385.0   356.0
                                                   Irregularly  5147.0                       2458.0  2689.0
                                                   Never         419.0                        155.0   264.0

### Languages

Tally supports multiple languages and crosstabs can apply labels from whatever languages were in the original dataset.

In [7]:
museum_dataset = tc.DataSet('museums')
museum_dataset.read_gvn('./data/Example_Museum.json', './data/Example_Museum.parquet')
museum_dataset.crosstab('museums', text_key='es-ES')
museum_dataset.crosstab('museums', text_key='en-GB')

Question                                                                          Total
Values                                                                            Total
Question                                           Values                              
museums. ¿Qué museos o galerías de arte ha visi... Base                           426.0
                                                   Museo Nacional de Ciencias     333.0
                                                   Museo del Diseño                92.0
                                                   Instituto de Textiles y Modas   47.0
                                                   Museo Arqueológico              26.0
                                                   Galeria Nacional de Arte        19.0
                                                   Galeria del Norte               21.0
                                                   Otra                            37.0
                                                   No respondió                     0.0

Question                                                                              Total
Values                                                                                Total
Question                                           Values                                  
museums. Which museums or art galleries have yo... Base                               426.0
                                                   National Museum of Science         333.0
                                                   Museum of Design                    92.0
                                                   Institute of Textiles and Fashion   47.0
                                                   Archeological Museum                26.0
                                                   National Art Gallery                19.0
                                                   Northern Gallery                    21.0
                                                   Other                               37.0
                                                   Not answered                         0.0

## Nesting crosstabs

Nesting crosstabs is done by adding the `>` signal to the `y` parameter (i.e. the banner/top). The nesting mechanism supports an arbitrarily deep level of nesting.

In [ ]:
dataset.crosstab('q1', 'gender > locality')

Question                                                                                                        gender. What is your gender?                                                                                                                            
Values                                                                                                                                  Male                                                                                       Female                               
Question                                                                       locality. How would you describe the areas in which you live?                                locality. How would you describe the areas in which you live?                               
Values                                                                                                       CBD (central business district)   Urban Suburban  Rural Remote                               CBD (central business district)   Urban Suburban  Rural Remote
Question                                Values                                                                                                                                                                                                                          
q1. What is your main fitness activity? Base                                                                               1567.0             1061.0    559.0  332.0  352.0                                             1539.0             1184.0    621.0  386.0  477.0
                                        Swimming                                                                             69.0               35.0     13.0    7.0   17.0                                               71.0               37.0     25.0    8.0    8.0
                                        Running/jogging                                                                      88.0               50.0     29.0   16.0   19.0                                               68.0               54.0     26.0   13.0   26.0
                                        Lifting weights                                                                     429.0              300.0    157.0   80.0  105.0                                              406.0              371.0    163.0   95.0  138.0
                                        Aerobics                                                                            615.0              371.0    201.0  125.0  100.0                                              585.0              396.0    234.0  145.0  171.0
                                        Yoga                                                                                 32.0               22.0     22.0    7.0   10.0                                               38.0               27.0     11.0   11.0    9.0
                                        Pilates                                                                              69.0               51.0     34.0   23.0   14.0                                               85.0               76.0     48.0   24.0   38.0
                                        Football (soccer)                                                                   163.0              125.0     62.0   34.0   52.0                                              152.0              116.0     69.0   53.0   46.0
                                        Basketball                                                                           18.0               20.0     11.0    9.0    6.0                                               26.0               15.0     13.0    6.0    6.0
                                        Hockey                                                                                1.0                1.0      1.0    0.0    0.0                                                1.0                0.0      0.0    0.0    0.0
                         

In [17]:
dataset.crosstab('q1', 'gender > q2b > q4 > locality')

Question                                                                                                                          gender. What is your gender?                                                                                                                        ...                                                                                                                                                                                                      
Values                                                                                                                                                    Male                                                                                                                        ...                                                                          Female                                                                                                                      
Question                                                                              q2b. How regularly do you participate in any fitness or sports activity?                                                                                                                        ...        q2b. How regularly do you participate in any fitness or sports activity?                                                                                                                      
Values                                                                                                                                               Regularly                                                                                                                        ...                                                                           Never                                                                                                                      
Question                                                                       q4. Do you ever participate in sports activities with people in your household?                                                                                                                        ... q4. Do you ever participate in sports activities with people in your household?                                                                                                                      
Values                                                                                                                                                     Yes                                                                                        No                              ...                                                                             Yes                                                                                        No                            
Question                                                                                         locality. How would you describe the areas in which you live?                             locality. How would you describe the areas in which you live?                              ...                   locality. How would you describe the areas in which you live?                             locality. How would you describe the areas in which you live?                            
Values                                                                                                                         CBD (central business district) Urban Suburban Rural Remote                               CBD (central business district) Urban Suburban Rural Remote  ...                                                 CBD (central business district) Urban Suburban Rural Remote                               CBD (central business district) Urban Suburban Rural Remote
Question                                Values                                                                                                                  

## Working with arrays (grids/loops)

Crosstabs also work with `arrays` (grids/loops). We use the museum dataset again.

In [12]:
museum_dataset.crosstab('rating.Column')

Question                               Total        rating.Column. Q30                                                                                           
Values                                  Base Not at all interested (1) Not particularly interested (2) No opinion (3) Slightly interested (4) Very interested (5)
Array              Questions                                                                                                                                     
rating.Column. Q30 Other                 0.0                       0.0                             0.0            0.0                     0.0                 0.0
                   Dinosaurs           258.0                      10.0                             1.0           27.0                   113.0               107.0
                   Conservation         56.0                       3.0                             2.0            7.0                    30.0                14.0
                   Fish and reptiles   111.0                       6.0                             5.0           28.0                    45.0                27.0
                   Fossils              65.0                       6.0                             1.0           15.0                    25.0                18.0
                   Birds                94.0                       9.0                             8.0           18.0                    32.0                27.0
                   Insects              79.0                      13.0                             2.0           18.0                    27.0                19.0
                   Whales               82.0                       2.0                             0.0           14.0                    28.0                38.0
                   Mammals             138.0                       4.0                             3.0           26.0                    55.0                50.0
                   Minerals             86.0                      13.0                             6.0           14.0                    25.0                28.0
                   Ecology              28.0                       3.0                             0.0            5.0                    10.0                10.0
                   Botany               55.0                      11.0                             3.0           13.0                    14.0                14.0
                   Origin of species   102.0                       4.0                             4.0           16.0                    29.0                49.0
                   Human biology       123.0                       4.0                             5.0           17.0                    34.0                63.0
                   Evolution           117.0                      12.0                             2.0           18.0                    41.0                44.0
                   Wildlife in danger   32.0                       0.0                             1.0            2.0                    16.0                13.0